In [ ]:
import pandas as pd
import wandb

%config Completer.use_jedi = False

In [ ]:
api = wandb.Api()
entity, project = "INPUT_YOUR_ENTITY", "curl"
runs = api.runs(entity + "/" + project) 

In [ ]:
dataset = ["wiki3029", "cifar10", "cifar100"]
records = []

for run in runs: 
    if "hydra_path" in run.config and run.config["dataset.name"] in dataset and run.config["name"] == "contrastive" and:
        records.append(run.config)


In [ ]:
df = pd.DataFrame.from_records(records)

In [ ]:
df.head(2)

In [ ]:
postfix = "contrastive_wights_path.txt"

seeds = set(df.seed.values)

In [ ]:
# wiki
for seed in seeds:

    wiki = df[(df["dataset.name"] == "wiki3029") & (df["seed"] == seed)]

    # Ks=4, Cs=4, lrs=4
    param_config = set()
    for row in wiki[["optimizer.lr", "dataset.num_used_classes", "loss.neg_size"]].values:
        param_config.add("-".join(map(str, row)))

    assert len(param_config) == 4 * 4 * 4
    
    for k in ["optimizer.lr", "dataset.num_used_classes", "loss.neg_size"]:
        assert len(set(wiki[k])) == 4

    with open("../jobs/language/wiki3029/seed_{}-{}".format(seed, postfix), "w") as f:
        for p in sorted(wiki.hydra_path.values):
            f.write(f"{p}/\n")


In [ ]:
# cifar10/100
for dname in ["cifar10", "cifar100"]:

    for seed in seeds:
        
#         if seed != 7:
#             continue
            
        cifar_df = df[(df["dataset.name"] == dname) & (df["seed"] == seed)]

        # lrs=3, C=1, Ks=4
        param_config = set()
        for row in cifar_df[["optimizer.lr", "dataset.num_used_classes", "loss.neg_size"]].values:
            param_config.add("-".join(map(str, row)))

#         assert len(param_config) == 3 * 1 * 4

#         for k, unique_num in zip(["optimizer.lr", "dataset.num_used_classes", "loss.neg_size"], [3, 1, 4]):
#             assert len(set(cifar_df[k])) == unique_num


        with open("../jobs/vision/{}/seed_{}-{}".format(dname, seed, postfix), "w") as f:
            for p in sorted(cifar_df.hydra_path.values):
                f.write(f"{p}/\n")
